<!--
 Copyright 2025 beedi.goua_square-ma
 
 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at
 
     https://www.apache.org/licenses/LICENSE-2.0
 
 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
-->



In [1]:
# 1. Imports et configuration
from pathlib import Path
import sys
base_dir = Path().resolve().parent
sys.path.append(str(base_dir))
base_dir


WindowsPath('C:/Users/beedi.goua_square-ma/Desktop/Gheb/projet perso/brouillon/budget-ai-dashboard/PROJET')

In [2]:
# 2. Affichage du répertoire de base
print(f"Répertoire de base : {base_dir}")
# 3. Vérification de l'existence du répertoire de base
print(base_dir.exists())

Répertoire de base : C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\brouillon\budget-ai-dashboard\PROJET
True


In [3]:
# 4. Ajout des chemins nécessaires au sys.path

sys.path.append(str(base_dir / "src"))
sys.path.append(str(base_dir / "data"))
sys.path.append(str(base_dir / "notebooks"))

In [4]:
# 5. Import des modules nécessaires
import pandas as pd
from src.features_engineering import (
    extract_date_features, encode_day_of_week_cyclic,
    encode_hour_cyclic, create_transaction_flags,
    calculate_saving_rate, clean_description,
    categorize_amount_level, tag_fixed_expenses
)


In [5]:
# 6. Chargement des données
data_dir = base_dir / "data"
df_path = data_dir / "transactions.csv"
df = pd.read_csv(df_path)


In [6]:
# analyse des données
print(f"Nombre de lignes dans le DataFrame : {len(df)}")
print(f"Colonnes du DataFrame : {df.columns.tolist()}")
# 7. Affichage des premières lignes du DataFrame
print("Premières lignes du DataFrame :")
print(df.head())


Nombre de lignes dans le DataFrame : 472
Colonnes du DataFrame : ['Transaction Date', 'Post Date', 'Description', 'Category', 'Type', 'Amount', 'Memo']
Premières lignes du DataFrame :
  Transaction Date   Post Date            Description      Category  Type  \
0       12/24/2023  12/24/2023        CHIPOTLE ONLINE  Food & Drink  Sale   
1       12/22/2023  12/24/2023        BRAVO-DEERFIELD  Food & Drink  Sale   
2       12/22/2023  12/24/2023        MARSHALLS #1172      Shopping  Sale   
3       12/22/2023  12/24/2023  DD DOORDASH WINGSANDR  Food & Drink  Sale   
4       12/21/2023  12/24/2023                 KROGER     Groceries  Sale   

   Amount  Memo  
0  -33.17   NaN  
1  -50.17   NaN  
2  -49.06   NaN  
3  -62.31   NaN  
4 -131.72   NaN  


In [7]:
df

,Transaction Date,Post Date,Description,Category,Type,Amount,Memo
0,12/24/2023,12/24/2023,CHIPOTLE ONLINE,Food & Drink,Sale,-33.17,NaN
1,12/22/2023,12/24/2023,BRAVO-DEERFIELD,Food & Drink,Sale,-50.17,NaN
2,12/22/2023,12/24/2023,MARSHALLS #1172,Shopping,Sale,-49.06,NaN
3,12/22/2023,12/24/2023,DD DOORDASH WINGSANDR,Food & Drink,Sale,-62.31,NaN
4,12/21/2023,12/24/2023,KROGER,Groceries,Sale,-131.72,NaN
...,...,...,...,...,...,...,...
467,9/26/2023,9/26/2023,Disney Plus,Bills & Utilities,Sale,-5.76,NaN
468,9/25/2023,9/26/2023,THE HONEST COMPANY INC.,Shopping,Sale,-11.87,NaN
469,9/23/2023,9/25/2023,McDonalds 39386,Food & Drink,Sale,-1.38,NaN
470,9/24/2023,9/24/2023,CHIPOTLE ONLINE,Food & Drink,Sale,-65.73,NaN


## Resampling
Notre but est d **augmenter artificiellement ton nombre d'observations** (passer d’environ 470 à 2000 lignes) tout en **respectant les structures importantes** de ton jeu de données, notamment :

* la répartition des **catégories** de dépenses,
* le **type** d’opération (achat, crédit, etc.),
* la **répartition temporelle** (mois, dates),
* la **variété dans les lignes** (éviter de dupliquer naïvement les mêmes lignes plusieurs fois).

### Méthode utilisée

1. **Stratification** :
   on a créé une clé de regroupement (`StratKey`) combinant les colonnes `Category`, `Type` et `Month`. Cette clé te permet de **préserver la distribution des types de transactions dans le temps**. L’échantillonnage se fait ensuite dans chaque groupe, à une fréquence proportionnelle à sa taille d’origine. Cela garantit une représentation équilibrée de chaque type de transaction.

2. **Resampling intelligent** :
   Pour chaque groupe, on effectue un échantillonnage :

   * **avec remise** si le groupe est trop petit,
   * **sans remise** sinon.
     Cela assure que **chaque catégorie existe en proportion réaliste** dans le nouveau DataFrame.

3. **Mélange et réattribution de dates** :
   Après avoir concaténé les échantillons, on  mélange les lignes pour éviter les répétitions visibles, puis on **réattribues des dates réalistes**. Cette réattribution respecte la fréquence originale des dates : par exemple, si beaucoup de transactions ont eu lieu en décembre, le nouveau dataset reflétera cette tendance. on évite ainsi les biais temporels artificiels.

---

### Avantages de ta méthode

* **Préservation des proportions réelles** entre catégories et types de transaction.
* **Répartition temporelle cohérente**, sans créer de "pics" artificiels.
* **Variété et réalisme** des données augmentées, propices à l’apprentissage machine ou à la génération de dashboards fiables.



In [8]:
df = df.copy()
df['Date'] = pd.to_datetime(df['Transaction Date'])

# Ajout de colonnes temporelles
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# On crée une clé de stratification
df['StratKey'] = (
    df['Category'].astype(str) + "_" +
    df['Type'].astype(str) + "_" +
    df['Month'].astype(str)
)


In [9]:

# Vérifions les valeurs manquantes dans la colonne 'Category'
missing_categories = df['Category'].isna().sum()

# Supprimons les lignes sans catégorie pour pouvoir faire l'échantillonnage stratifié
df = df.dropna(subset=['Category'])


In [10]:

from sklearn.utils import resample
import numpy as np
# 8. Échantillonnage stratifié
# Échantillonnage stratifié pour obtenir un DataFrame de taille désirée
desired_size = 2000
multiplier = int(np.ceil(desired_size / len(df)))




In [11]:

resampled_df = pd.DataFrame()

# Répéter chaque groupe selon sa fréquence relative
for key, group in df.groupby('StratKey'):
    proportion = len(group) / len(df)
    target_rows = int(desired_size * proportion)

    # Si le groupe est petit, on fait un tirage avec remise
    sampled = group.sample(n=target_rows, replace=(len(group) < target_rows), random_state=42)
    resampled_df = pd.concat([resampled_df, sampled], ignore_index=True)


In [12]:

# Réinitialisation de l'index
resampled_df = resampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
# Réattribution des dates
dates_freq = df['Transaction Date'].value_counts(normalize=True)
new_dates = np.random.choice(dates_freq.index, size=len(resampled_df), p=dates_freq.values)

# Conversion des nouvelles dates en datetime et réorganisation du DataFrame
resampled_df['Transaction Date'] = pd.to_datetime(new_dates)
resampled_df = resampled_df.sort_values("Transaction Date").reset_index(drop=True)


In [13]:
print("Ancienne répartition :")
print(df['StratKey'].value_counts(normalize=True).head())

print("Nouvelle répartition :")
print(resampled_df['StratKey'].value_counts(normalize=True).head())

print("Taille finale :", len(resampled_df))


Ancienne répartition :
StratKey
Food & Drink_Sale_10    0.135484
Food & Drink_Sale_11    0.113978
Shopping_Sale_11        0.113978
Food & Drink_Sale_12    0.088172
Shopping_Sale_12        0.086022
Name: proportion, dtype: float64
Nouvelle répartition :
StratKey
Food & Drink_Sale_10    0.136364
Food & Drink_Sale_11    0.114646
Shopping_Sale_11        0.114646
Food & Drink_Sale_12    0.088889
Shopping_Sale_12        0.086869
Name: proportion, dtype: float64
Taille finale : 1980


In [14]:
resampled_df.drop(columns=['StratKey', 'Month', 'Year'], inplace=True)


In [15]:
df

,Transaction Date,Post Date,Description,Category,Type,Amount,Memo,Date,Month,Year,StratKey
0,12/24/2023,12/24/2023,CHIPOTLE ONLINE,Food & Drink,Sale,-33.17,NaN,2023-12-24,12,2023,Food & Drink_Sale_12
1,12/22/2023,12/24/2023,BRAVO-DEERFIELD,Food & Drink,Sale,-50.17,NaN,2023-12-22,12,2023,Food & Drink_Sale_12
2,12/22/2023,12/24/2023,MARSHALLS #1172,Shopping,Sale,-49.06,NaN,2023-12-22,12,2023,Shopping_Sale_12
3,12/22/2023,12/24/2023,DD DOORDASH WINGSANDR,Food & Drink,Sale,-62.31,NaN,2023-12-22,12,2023,Food & Drink_Sale_12
4,12/21/2023,12/24/2023,KROGER,Groceries,Sale,-131.72,NaN,2023-12-21,12,2023,Groceries_Sale_12
...,...,...,...,...,...,...,...,...,...,...,...
467,9/26/2023,9/26/2023,Disney Plus,Bills & Utilities,Sale,-5.76,NaN,2023-09-26,9,2023,Bills & Utilities_Sale_9
468,9/25/2023,9/26/2023,THE HONEST COMPANY INC.,Shopping,Sale,-11.87,NaN,2023-09-25,9,2023,Shopping_Sale_9
469,9/23/2023,9/25/2023,McDonalds 39386,Food & Drink,Sale,-1.38,NaN,2023-09-23,9,2023,Food & Drink_Sale_9
470,9/24/2023,9/24/2023,CHIPOTLE ONLINE,Food & Drink,Sale,-65.73,NaN,2023-09-24,9,2023,Food & Drink_Sale_9


In [16]:
resampled_df

,Transaction Date,Post Date,Description,Category,Type,Amount,Memo,Date
0,2023-09-23,11/14/2023,LOVELAND KIDS CHILDCARE,Personal,Sale,-305.00,NaN,2023-11-13
1,2023-09-23,10/23/2023,DD DOORDASH PIZZAATTH,Food & Drink,Sale,-40.70,NaN,2023-10-22
2,2023-09-23,11/26/2023,ZAP*ZAPPOS.COM,Shopping,Return,30.53,NaN,2023-11-24
3,2023-09-24,11/2/2023,PANERA BREAD #204922 O,Food & Drink,Sale,-9.98,NaN,2023-11-02
4,2023-09-24,11/9/2023,DD DOORDASH CATCH-A-F,Food & Drink,Sale,-33.45,NaN,2023-11-08
...,...,...,...,...,...,...,...,...
1975,2023-12-23,12/24/2023,DD DOORDASH WINGSANDR,Food & Drink,Sale,-62.31,NaN,2023-12-22
1976,2023-12-23,10/6/2023,PANERA BREAD #204922 O,Food & Drink,Sale,-10.41,NaN,2023-10-06
1977,2023-12-23,12/10/2023,SHELL OIL 57445097702,Gas,Sale,-32.26,NaN,2023-12-07
1978,2023-12-24,10/26/2023,FIVE STAR FOOD,Food & Drink,Sale,-3.15,NaN,2023-10-25


In [17]:
# Ancienne
print(df['Category'].value_counts(normalize=True).sort_index())

# Nouvelle
print(resampled_df['Category'].value_counts(normalize=True).sort_index())


Category
Automotive            0.006452
Bills & Utilities     0.015054
Education             0.002151
Entertainment         0.017204
Fees & Adjustments    0.019355
Food & Drink          0.363441
Gas                   0.118280
Groceries             0.079570
Health & Wellness     0.015054
Personal              0.038710
Shopping              0.303226
Travel                0.021505
Name: proportion, dtype: float64
Category
Automotive            0.006061
Bills & Utilities     0.014141
Education             0.002020
Entertainment         0.017172
Fees & Adjustments    0.018687
Food & Drink          0.365657
Gas                   0.118687
Groceries             0.078788
Health & Wellness     0.014646
Personal              0.038384
Shopping              0.305051
Travel                0.020707
Name: proportion, dtype: float64


<!--
 Copyright 2025 beedi.goua_square-ma
 
 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at
 
     https://www.apache.org/licenses/LICENSE-2.0
 
 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
-->



In [18]:
"""for col in df.columns:
    if col in resampled_df.columns:
        if set(df[col].dropna().unique()) != set(resampled_df[col].dropna().unique()):
            print(f"Changement détecté dans la colonne '{col}'")
            

"""

'for col in df.columns:\n    if col in resampled_df.columns:\n        if set(df[col].dropna().unique()) != set(resampled_df[col].dropna().unique()):\n            print(f"Changement détecté dans la colonne \'{col}\'")\n\n\n'

In [19]:

"""for col in df.columns:
    if col in resampled_df.columns and df[col].dtype == 'object':
        orig_dist = df[col].value_counts(normalize=True)
        new_dist = resampled_df[col].value_counts(normalize=True)
        combined_index = orig_dist.index.union(new_dist.index)
        diff = (orig_dist.reindex(combined_index, fill_value=0) - new_dist.reindex(combined_index, fill_value=0)).abs()
        if diff.max() > 0.05:  # seuil de tolérance
            print(f"Distribution changée pour la colonne '{col}' (diff max : {diff.max():.2f})")
"""

'for col in df.columns:\n    if col in resampled_df.columns and df[col].dtype == \'object\':\n        orig_dist = df[col].value_counts(normalize=True)\n        new_dist = resampled_df[col].value_counts(normalize=True)\n        combined_index = orig_dist.index.union(new_dist.index)\n        diff = (orig_dist.reindex(combined_index, fill_value=0) - new_dist.reindex(combined_index, fill_value=0)).abs()\n        if diff.max() > 0.05:  # seuil de tolérance\n            print(f"Distribution changée pour la colonne \'{col}\' (diff max : {diff.max():.2f})")\n'

In [20]:
df["Category"].unique()

array(['Food & Drink', 'Shopping', 'Groceries', 'Gas', 'Automotive',
       'Health & Wellness', 'Personal', 'Travel', 'Bills & Utilities',
       'Fees & Adjustments', 'Entertainment', 'Education'], dtype=object)

In [21]:


data = resampled_df.copy()
#7. Traitement des données
data = extract_date_features(data)
data = encode_day_of_week_cyclic(data)
data = encode_hour_cyclic(data)
data = clean_description(data)
data = create_transaction_flags(data)
data = categorize_amount_level(data)
data = tag_fixed_expenses(data)


In [22]:
df.columns

Index(['Transaction Date', 'Post Date', 'Description', 'Category', 'Type',
       'Amount', 'Memo', 'Date', 'Month', 'Year', 'StratKey'],
      dtype='object')

In [23]:
data.columns

Index(['Transaction Date', 'Post Date', 'Description', 'Category', 'Type',
       'Amount', 'Memo', 'Date', 'Year', 'Month', 'Week', 'Day', 'Hour',
       'DayOfWeek', 'IsWeekend', 'IsMonthEnd', 'DayOfWeek_sin',
       'DayOfWeek_cos', 'Hour_sin', 'Hour_cos', 'Cleaned_Description',
       'IsLargeAmount', 'IsVeryLargeAmount', 'AmountLevel', 'IsRecurring'],
      dtype='object')

In [24]:

# 4. Calcul taux d’épargne
monthly_saving = calculate_saving_rate(data)



In [25]:

# 5. Vérification
display(data.head())
display(monthly_saving)



,Transaction Date,Post Date,Description,Category,Type,Amount,Memo,Date,Year,Month,...,IsMonthEnd,DayOfWeek_sin,DayOfWeek_cos,Hour_sin,Hour_cos,Cleaned_Description,IsLargeAmount,IsVeryLargeAmount,AmountLevel,IsRecurring
0,2023-09-23,11/14/2023,LOVELAND KIDS CHILDCARE,Personal,sale,-305.00,NaN,2023-11-13,2023,11,...,0,0.000000,1.000000,0.0,1.0,loveland kids childcare,0,0,small,1
1,2023-09-23,10/23/2023,DD DOORDASH PIZZAATTH,Food & Drink,sale,-40.70,NaN,2023-10-22,2023,10,...,0,-0.781831,0.623490,0.0,1.0,dd doordash pizzaatth,0,0,medium,1
2,2023-09-23,11/26/2023,ZAP*ZAPPOS.COM,Shopping,return,30.53,NaN,2023-11-24,2023,11,...,0,-0.433884,-0.900969,0.0,1.0,zapzapposcom,1,1,very_large,1
3,2023-09-24,11/2/2023,PANERA BREAD #204922 O,Food & Drink,sale,-9.98,NaN,2023-11-02,2023,11,...,0,0.433884,-0.900969,0.0,1.0,panera bread 204922 o,0,0,large,1
4,2023-09-24,11/9/2023,DD DOORDASH CATCH-A-F,Food & Drink,sale,-33.45,NaN,2023-11-08,2023,11,...,0,0.974928,-0.222521,0.0,1.0,dd doordash catchaf,0,0,medium,1


Type,Year,Month,adjustment,fee,return,sale,SavingRate
0,2023,9,0.00,0.0,0.00,-6477.01,0.0
1,2023,10,1200.00,-196.0,25.52,-30945.60,0.0
2,2023,11,104.45,0.0,811.61,-30662.98,0.0
3,2023,12,91.54,0.0,0.00,-17394.15,0.0


In [ ]:
print(data_dir.exists())


True


In [35]:
# 6. Sauvegarde
output_dir = base_dir / "outputs"
output_dir.mkdir(parents=True, exist_ok=True)
sys.path.append(str(output_dir))
print(output_dir.exists())


True


In [36]:
data.to_csv(output_dir / "transactions_enriched.csv", index=False)
monthly_saving.to_csv(output_dir / "monthly_saving.csv", index=False)